# GNOD Part 3

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
secrets_file = open("secrets.txt","r")

In [ ]:
string = secrets_file.read()

In [ ]:
#string.split('\n')

In [ ]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [ ]:
#secrets_dict

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['ClientID'],
                                                           client_secret=secrets_dict['Clientsecret']))

In [ ]:
playlist = sp.user_playlist_tracks("spotify", "5S8SJdl1BDc0ugpkEvFsIL")

In [ ]:
playlist 

In [ ]:
playlist.keys() 

In [ ]:
len(playlist["items"])

In [ ]:
playlist["total"]

In [ ]:
playlist['items'][0].keys() 

In [ ]:
playlist['items'][0]['track'].keys() 

In [ ]:
playlist['items'][0]['track']['artists']

In [ ]:
playlist['items'][0]['track']['name']

In [ ]:
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"], track['uri']) for artist in track["artists"]]

In [ ]:
my_track = playlist['items'][0]['track']

In [ ]:
song_info = get_name_artists_from_track(my_track)
song_info

In [ ]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000)
    return tracks

In [ ]:
all_tracks = get_playlist_tracks("5S8SJdl1BDc0ugpkEvFsIL")
#len(all_tracks)

In [ ]:
len(all_tracks)

In [ ]:
all_songs_artists = []

for item in all_tracks:
    track = item['track']
    song_artists = get_name_artists_from_track(track)
    uri = get_name_artists_from_track(track)
    all_songs_artists.append(song_artists)

all_songs_artists

In [ ]:
len(all_songs_artists)

In [ ]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [ ]:
flatten_songs = flatten(all_songs_artists)
flatten_songs

In [ ]:
len(flatten_songs)

# DataFrame creation

In [ ]:
import pandas as pd
playlist_mix = pd.DataFrame((flatten_songs), columns=['title','artist','uri'])
playlist_mix

# 5,000 songs - Playlist 1

In [ ]:
playlist_mix1 = playlist_mix[:5000]
playlist_mix1

In [ ]:
import numpy as np
import pandas as pd

uris = playlist_mix1['uri'].tolist()
uri_groups = np.array_split(uris, len(uris) // 100 + 1)

In [ ]:
len(uri_groups)

In [ ]:
audio_features_list = []

for uri_group in uri_groups:
    audio_features = sp.audio_features(uri_group)
    audio_features_list.extend(audio_features)
    sleep(randint(1000, 3000) / 1000)

audio_features_list

In [ ]:
audio_features_df = pd.DataFrame(audio_features_list)

playlist_mix1 = pd.concat([playlist_mix1, audio_features_df], axis=1)

In [ ]:
playlist_mix1 = playlist_mix1.drop_duplicates()

In [ ]:
playlist_mix1.columns

In [ ]:
len(playlist_mix1)